In [14]:
#Clean notebook for data clean up explantions

In [15]:
#imported our Dependancies as needed
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import numpy as np
import requests
import json
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

#Importing CSV files found online
#https://data.world/liz-friedman/us-covid-19-data-from-nytimes
#https://data.world/qventus/covid-19-localized-scenario-planner
state_lockdown_file = pd.read_csv("data_sources/lockdown_us.csv")
state_deaths_cases_file = pd.read_csv("data_sources/liz-friedman-us-covid-19-data-from-nytimes/us-states.csv")
county_population_file = pd.read_csv("data_sources/qventus-covid-19-localized-scenario-planner/qventus-covid-19-localized-scenario-planner/covid_county_population_usafacts.csv")

In [40]:
#URL pulls state data just for the current or last weekday reported
url = "https://api.covidtracking.com/v1/states/current.json"
state_current_data = (requests.get(url)).json()
#Creating dataframe form api request
state_current_data=pd.DataFrame(state_current_data)
#date format is yyyymmdd this line changes to yyyy-mm-dd
state_current_data['date'] = pd.to_datetime(state_current_data['date'], format='%Y%m%d')

#Dropping US territoires from state current data
state_current_data = state_current_data.set_index("state")
state_current_data = state_current_data.drop(["AS","GU","MP","PR", "VI","DC"])
state_current_data = state_current_data.reset_index()

state_current_data.head(50)

,state,date,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,AK,2020-08-31,6125,342505,NaN,39.0,NaN,NaN,NaN,8.0,...,348630,0,0,cf39874094a3ac9c4c3b69e94b68927b52a862eb,0,0,0,0,0,
1,AL,2020-08-31,126058,859229,NaN,1004.0,14267.0,NaN,1474.0,NaN,...,985287,20,0,7547348e99fc537845e193fe2c58d552b40036cb,0,0,0,0,0,
2,AR,2020-08-31,61224,665811,NaN,420.0,4213.0,NaN,NaN,87.0,...,727035,13,31,6a343a185910575b6b3dc7204110439160baf877,0,0,0,0,0,
3,AZ,2020-08-31,201835,1002594,NaN,768.0,21405.0,256.0,NaN,152.0,...,1204429,-1,-16,3410711ea86edea436f3bf920d6fef2b765a927c,0,0,0,0,0,
4,CA,2020-08-31,704085,10669220,NaN,4829.0,NaN,1346.0,NaN,NaN,...,11373305,28,0,3772a213d9eba8933d8d80c678cdbb3496a8a730,0,0,0,0,0,
5,CO,2020-08-31,57223,652673,NaN,240.0,7010.0,NaN,NaN,NaN,...,709896,0,17,065dff9892e53e1091a35af6ee073d17bd037f86,0,0,0,0,0,
6,CT,2020-08-31,52879,1120217,NaN,52.0,11180.0,NaN,NaN,NaN,...,1173096,0,0,9703ebeff3e3fa39cc3ddcf6d8199adb02a30621,0,0,0,0,0,
7,DE,2020-08-31,17429,219420,NaN,58.0,NaN,16.0,NaN,NaN,...,236849,1,0,7b665799ed78f58a5fe8c4dd4499ce295b5e4bba,0,0,0,0,0,
8,FL,2020-08-31,623471,3992068,3472.0,3736.0,38942.0,NaN,NaN,NaN,...,4615539,68,85,800b088f3e86a654f7a0c8641f0e9d2e9c3c0b32,0,0,0,0,0,
9,GA,2020-08-31,270471,2073163,NaN,2463.0,24604.0,NaN,4493.0,NaN,...,2343634,28,32,60b48955646ed5173da7cd102d87378d6d03dbec,0,0,0,0,0,


In [37]:
#URL pulls collection of all states current data from previous dates
url = "https://api.covidtracking.com/v1/states/daily.json"
state_daily_data = (requests.get(url)).json()

#Creating dataframe form API request
state_daily_data=pd.DataFrame(state_daily_data)

#date format is yyyymmdd this line changes to yyyy-mm-dd
#Used later in adding specific cloumns for "Month" and "Day" for sorting data by timeframe
state_daily_data['date'] = pd.to_datetime(state_daily_data['date'], format='%Y%m%d')

#Dropping US Territories from state list
state_daily_data = state_daily_data.set_index("state")
state_daily_data = state_daily_data.drop(["AS","GU","MP","PR", "VI","DC"])
state_daily_data = state_daily_data.reset_index()

#loop for summing for total state cases
states = state_daily_data["state"].unique()
total_cases_by_state = []

for state in states:
    state_case = state_daily_data.loc[state_daily_data["state"] == state]
    total_cases_by_state.append(state_case["positive"].sum())

state_daily_data.head(500)

,state,date,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,AK,2020-08-31,6125.0,342505.0,NaN,39.0,NaN,NaN,NaN,8.0,...,348630,0,0,cf39874094a3ac9c4c3b69e94b68927b52a862eb,0,0,0,0,0,
1,AL,2020-08-31,126058.0,859229.0,NaN,1004.0,14267.0,NaN,1474.0,NaN,...,985287,20,0,7547348e99fc537845e193fe2c58d552b40036cb,0,0,0,0,0,
2,AR,2020-08-31,61224.0,665811.0,NaN,420.0,4213.0,NaN,NaN,87.0,...,727035,13,31,6a343a185910575b6b3dc7204110439160baf877,0,0,0,0,0,
3,AZ,2020-08-31,201835.0,1002594.0,NaN,768.0,21405.0,256.0,NaN,152.0,...,1204429,-1,-16,3410711ea86edea436f3bf920d6fef2b765a927c,0,0,0,0,0,
4,CA,2020-08-31,704085.0,10669220.0,NaN,4829.0,NaN,1346.0,NaN,NaN,...,11373305,28,0,3772a213d9eba8933d8d80c678cdbb3496a8a730,0,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,VT,2020-08-22,1553.0,117051.0,NaN,15.0,NaN,NaN,NaN,NaN,...,118604,0,0,2b07a9ebfecb66d48a3ced58b676057cea9f811e,0,0,0,0,0,
496,WA,2020-08-22,71598.0,1304935.0,NaN,371.0,6469.0,NaN,NaN,45.0,...,1376533,13,69,2cd6b0485b67923c09e6c83addc4ae226e549bc6,0,0,0,0,0,
497,WI,2020-08-22,74726.0,1110788.0,190.0,350.0,5545.0,119.0,986.0,NaN,...,1185514,14,40,641ab4831d3f4d09a4f230419330ce5f5acbb874,0,0,0,0,0,
498,WV,2020-08-22,9185.0,382673.0,NaN,138.0,NaN,51.0,NaN,23.0,...,391858,6,0,adacd3b5fc05e71b711ebad41b3f917a35dd538c,0,0,0,0,0,


In [18]:
#loop for finding positive rate per month
#Positive cases per month / total of positive and negitive test results
months = [1,2,3,4,5,6,7,8]
positive_rates = []
state_list = []
month_list = []

for state in states:
    
    single_state = state_daily_data.loc[state_daily_data["state"] == state]
    single_state['date'] = pd.to_datetime(single_state['date'], format='%Y%m%d')    
    single_state = single_state.sort_values("date")
    single_state['month'] = pd.DatetimeIndex(single_state['date']).month
    single_state['day'] = pd.DatetimeIndex(single_state['date']).day
    for month in months:
        print(state)
        try:
            single_month = single_state.loc[single_state["month"] == month]
            single_month = single_month.reset_index()
            first_row = single_month.iloc[0]
            last_row = single_month.iloc[-1]
            month_pos = int(last_row["positive"]) - int(first_row["positive"])
            month_tot = int(last_row["posNeg"]) - int(first_row["posNeg"])
            positive_rate = month_pos / month_tot
            positive_rate = "{:.2%}".format(positive_rate)
            positive_rates.append(positive_rate)
            state_list.append(state)
            month_list.append(month)
            
            
        except IndexError:
            #some states had no cases in the first 2 months
            print("no data for this month")
        except ValueError:
            #New Jersey had had values of NaN causing a value error
            print("value issue")
        except ZeroDivisionError:
            #Washington first month total comes out to 0 causing a division by zero error
            print("month total = 0")

AK
no data for this month
AK
no data for this month
AK
AK
AK
AK
AK
AK
AL
no data for this month
AL
no data for this month
AL
AL
AL
AL
AL
AL
AR
no data for this month
AR
no data for this month
AR
AR
AR
AR
AR
AR
AZ
no data for this month
AZ
no data for this month
AZ
AZ
AZ
AZ
AZ
AZ
CA
no data for this month
CA
no data for this month
CA
CA
CA
CA
CA
CA
CO
no data for this month
CO
no data for this month
CO
CO
CO
CO
CO
CO
CT
no data for this month
CT
no data for this month
CT
CT
CT
CT
CT
CT
DE
no data for this month
DE
no data for this month
DE
DE
DE
DE
DE
DE
FL
no data for this month
FL
no data for this month
FL
FL
FL
FL
FL
FL
GA
no data for this month
GA
no data for this month
GA
GA
GA
GA
GA
GA
HI
no data for this month
HI
no data for this month
HI
HI
HI
HI
HI
HI
IA
no data for this month
IA
no data for this month
IA
IA
IA
IA
IA
IA
ID
no data for this month
ID
no data for this month
ID
ID
ID
ID
ID
ID
IL
no data for this month
IL
no data for this month
IL
IL
IL
IL
IL
IL
IN
no data for this 

In [44]:
#Dataframe created from above loop
pos_rates_df = pd.DataFrame({
    "State": state_list,
    "Positive Rate": positive_rates,
    "Month": month_list
})

pos_rates_df.head(30)

,State,Positive Rate,Month
0,AK,3.21%,3
1,AK,1.53%,4
2,AK,0.22%,5
3,AK,0.82%,6
4,AK,2.27%,7
5,AK,2.09%,8
6,AL,13.48%,3
7,AL,7.48%,4
8,AL,8.55%,5
9,AL,10.75%,6


In [20]:
#States total population by summing all counties population
state_pop = []
states = county_population_file["State"].unique()

for state in states:
    state_df = county_population_file.loc[county_population_file["State"] == state]
    state_pop.append(state_df["population"].sum())
    
population_df = pd.DataFrame({
    "state": states,
    "population": state_pop
})

In [21]:
#State deaths cases data imported from sources folder
state_deaths_cases_file.head()

#Sorting alphabetically by state
state_deaths_cases_file = state_deaths_cases_file.rename(columns={"state":"State"})
state_deaths_cases_file = state_deaths_cases_file.sort_values("State")

#Breaking down date into a month and day column for easier filtering by month
state_deaths_cases_file['month'] = pd.DatetimeIndex(state_deaths_cases_file['date']).month
state_deaths_cases_file['day'] = pd.DatetimeIndex(state_deaths_cases_file['date']).day

#Dropping any empty rows from dataframe
state_deaths_cases_file = state_deaths_cases_file.dropna()
state_deaths_cases_file = state_deaths_cases_file.set_index("State")

state_deaths_cases_file.head()

,date,fips,cases,deaths,month,day
State,,,,,,
Alabama,2020-04-24,1,6026,209,4,24
Alabama,2020-07-18,1,65234,1286,7,18
Alabama,2020-05-07,1,9046,369,5,7
Alabama,2020-05-09,1,9668,390,5,9
Alabama,2020-08-18,1,110361,1936,8,18


In [22]:
#Looking at specific state data for states according to lockdown
positive_closings = state_deaths_cases_file.loc[["California","Florida","New York", 
                                                "Kansas","Texas","Washington"],
                                                ["date","cases","month"]]
positive_closings = positive_closings.sort_values("date")
positive_closings.head()

,date,cases,month
State,,,
Washington,2020-01-21,1,1
Washington,2020-01-22,1,1
Washington,2020-01-23,1,1
Washington,2020-01-24,1,1
Washington,2020-01-25,1,1


In [23]:
#Pulling data from specific state prior to the decsion to lock down.
#once data is pulled cases are summed up for each month
california = positive_closings.loc["California"]
cali_closings = california.loc[(california["date"] >= "2020-03-19"),:]
cali_closings = california.loc[(california["date"] <= "2020-05-12"),:]
cali_closings = cali_closings.groupby(by=["month"]).agg(sum)

kansas = positive_closings.loc["Kansas"]
ks_closings = kansas.loc[(kansas["date"]>= "2020-04-19"),:]
ks_closings = kansas.loc[(kansas["date"]<= "2020-05-03"),:]
ks_closings = ks_closings.groupby(by=["month"]).agg(sum)

florida = positive_closings.loc["Florida"]
fl_closings = florida.loc[(florida["date"]>= "2020-04-03"),:]
fl_closings = florida.loc[(florida["date"]<= "2020-04-30"),:]
fl_closings = fl_closings.groupby(by=["month"]).agg(sum)

new_york = positive_closings.loc["New York"]
ny_closings = new_york.loc[(new_york["date"]>= "2020-03-22"),:]
ny_closings = new_york.loc[(new_york["date"]<= "2020-05-15"),:]
ny_closings = ny_closings.groupby(by=["month"]).agg(sum)

washington = positive_closings.loc["Washington"]
wa_closings = washington.loc[(washington["date"]>= "2020-03-23"),:]
wa_closings = washington.loc[(washington["date"]<= "2020-05-04"),:]
wa_closings = wa_closings.groupby(by=["month"]).agg(sum)

texas = positive_closings.loc["Texas"]
tx_closings = texas.loc[(texas["date"]>= "2020-04-02"),:]
tx_closings = texas.loc[(texas["date"]<= "2020-04-30"),:]
tx_closings = tx_closings.groupby(by=["month"]).agg(sum)

In [24]:
#Sorting by states Re-open dates.  Data after the reopen is summed up to show total each month.
cali_reo = california.loc[(california["date"] >= "2020-05-12"),:]
cali_reo = cali_reo.groupby(by=["month"]).agg(sum)

ks_reo = kansas.loc[(kansas["date"]>= "2020-05-03"),:]
ks_reo = ks_reo.groupby(by=["month"]).agg(sum)

fl_reo = florida.loc[(florida["date"]>= "2020-04-30"),:]
fl_reo = fl_reo.groupby(by=["month"]).agg(sum)

ny_reo = new_york.loc[(new_york["date"]>= "2020-05-15"),:]
ny_reo = ny_reo.groupby(by=["month"]).agg(sum)

wa_reo = washington.loc[(washington["date"]>= "2020-05-04"),:]
wa_reo = wa_reo.groupby(by=["month"]).agg(sum)

tx_reo = texas.loc[(texas["date"]>= "2020-04-30"),:]
tx_reo = tx_reo.groupby(by=["month"]).agg(sum)

In [26]:
#Removing some of the counties listed and the types of stay at home/shelter in place


state_lockdown_file['month'] = pd.DatetimeIndex(state_lockdown_file['Date']).month
state_lockdown_file['day'] = pd.DatetimeIndex(state_lockdown_file['Date']).day

state_lockdown_file

,Country,State,County,Date,Type,month,day
0,United States,Alabama,NaN,2020-04-04,Stay at home,4,4
1,United States,Alabama,Birmingham,2020-03-24,Shelter in place,3,24
2,United States,Alaska,NaN,2020-03-28,Stay at home,3,28
3,United States,Arizona,NaN,2020-03-31,Stay at home,3,31
4,United States,California,NaN,2020-03-19,Stay at home,3,19
...,...,...,...,...,...,...,...
168,United States,Virginia,NaN,2020-03-30,Stay at home,3,30
169,United States,Washington,NaN,2020-03-23,Stay at home,3,23
170,United States,West Virginia,NaN,2020-03-24,Stay at home,3,24
171,United States,Wisconsin,NaN,2020-03-25,Stay at home,3,25


In [27]:
#Takeing data from the State current API
state_current_narrow = state_current_data[["state","positive","death","hospitalized"]]
state_current_drop = state_current_narrow.dropna()
state_current_drop
state_current_drop = state_current_drop.set_index('state')
state_current_drop

#Merging with the state population dataframe to compair positive cases and deaths with state total population
state_current_df = pd.merge(state_current_drop, population_df, on='state')

state_current_df.head()

,state,positive,death,hospitalized,population
0,AL,126058,2182,14267.0,4903185
1,AR,61224,797,4213.0,3017804
2,AZ,201835,5029,21405.0,7278717
3,CO,57223,1843,7010.0,5758736
4,CT,52879,4465,11180.0,3565287


In [29]:
#Making a new dataframe of 5 states with early lockdown date
states_early_lock = state_current_drop.loc[["KY","MA","NY","WA","WI"],["death","hospitalized"]]
states_early_lock

#Making dataframe of 5 states with a late lockdown date
states_late_lock = state_current_drop.loc[["FL","GA","KS","SC","AZ"],["death","hospitalized"]]
states_late_lock

,death,hospitalized
state,,
FL,11331,38942.0
GA,5632,24604.0
KS,446,2304.0
SC,2720,7811.0
AZ,5029,21405.0


In [47]:
#Selection of states for having summer tourist attractions
#to see if cases increased during summer months
summer_fun = state_deaths_cases_file.loc[["California","Florida","Hawaii",
                                                "Missouri","Texas","Nevada"],
                                                ["date","cases","month"]]

summer_fun.head()

,date,cases,month
State,,,
California,2020-04-27,45208,4
California,2020-03-18,893,3
California,2020-05-13,73218,5
California,2020-07-23,433175,7
California,2020-05-11,69514,5
...,...,...,...
Hawaii,2020-08-12,3935,8
Hawaii,2020-07-23,1468,7
Hawaii,2020-06-05,653,6
